In [1]:
import pandas as pd
df = pd.read_csv('../data/processed/indexed_price.csv')

In [2]:
df.head()

,SISTEMA,TARIFA,CIA,MES,FEE,P1,P2,P3,P4,P5,P6
0,BALEARES,2.0TD,ACCIONA,2023-06-01 00:00:00,Cierzo,0.212767,0.165975,0.142781,0.0,0.0,0.0
1,BALEARES,2.0TD,ACCIONA,2023-05-01 00:00:00,Cierzo,0.190597,0.143704,0.120055,0.0,0.0,0.0
2,BALEARES,2.0TD,ACCIONA,2023-04-01 00:00:00,Cierzo,0.190032,0.143135,0.119475,0.0,0.0,0.0
3,BALEARES,2.0TD,ACCIONA,2023-03-01 00:00:00,Cierzo,0.208748,0.161937,0.138661,0.0,0.0,0.0
4,BALEARES,2.0TD,ACCIONA,2023-02-01 00:00:00,Cierzo,0.260442,0.213867,0.191651,0.0,0.0,0.0


In [3]:
from datetime import datetime

df['MES'] = pd.to_datetime(df['MES'])

In [4]:
# df.drop(columns=['Unnamed: 0.1',	'Unnamed: 0'],inplace=True)
# df.to_csv('../data/processed/indexed_price.csv',index=False)

In [5]:
fecha_actual = datetime.now()

df['diferencia'] = (fecha_actual - df['MES']).abs() # diferencia entre cada fecha 'MES' y la fecha actual

condicion = (df['SISTEMA'] == 'PENINSULA') & (df['TARIFA'] == '2.0TD') # filtrar para obtener las filas más cercanas por CIA y con SISTEMA y TARIFA específicos
filas_mas_cercanas = df[condicion].groupby(['SISTEMA', 'TARIFA', 'CIA']).apply(lambda x: x[x['diferencia'] == x['diferencia'].min()])

filas_mas_cercanas = filas_mas_cercanas.drop('diferencia', axis=1).reset_index(drop=True)

print(filas_mas_cercanas)


      SISTEMA TARIFA      CIA        MES        FEE        P1        P2  \
0   PENINSULA  2.0TD  ACCIONA 2023-07-01     Cierzo  0.209144  0.162335   
1   PENINSULA  2.0TD  ACCIONA 2023-07-01    Levante  0.228429  0.181620   
2   PENINSULA  2.0TD  ACCIONA 2023-07-01   Levante+  0.228429  0.181620   
3   PENINSULA  2.0TD  ACCIONA 2023-07-01   Poniente  0.218279  0.171470   
4   PENINSULA  2.0TD  ACCIONA 2023-07-01  Poniente+  0.218279  0.171470   
..        ...    ...      ...        ...        ...       ...       ...   
64  PENINSULA  2.0TD      MAX 2023-07-01         30       NaN       NaN   
65  PENINSULA  2.0TD      MAX 2023-07-01         35       NaN       NaN   
66  PENINSULA  2.0TD      MAX 2023-07-01         40       NaN       NaN   
67  PENINSULA  2.0TD      MAX 2023-07-01          6       NaN       NaN   
68  PENINSULA  2.0TD      MAX 2023-07-01          8       NaN       NaN   

          P3   P4   P5   P6  
0   0.139067  0.0  0.0  0.0  
1   0.158352  0.0  0.0  0.0  
2   0.158

In [6]:
pd.set_option('display.max_rows', None)

In [7]:
filas_mas_cercanas.head()

,SISTEMA,TARIFA,CIA,MES,FEE,P1,P2,P3,P4,P5,P6
0,PENINSULA,2.0TD,ACCIONA,2023-07-01,Cierzo,0.209144,0.162335,0.139067,0.0,0.0,0.0
1,PENINSULA,2.0TD,ACCIONA,2023-07-01,Levante,0.228429,0.181620,0.158352,0.0,0.0,0.0
2,PENINSULA,2.0TD,ACCIONA,2023-07-01,Levante+,0.228429,0.181620,0.158352,0.0,0.0,0.0
3,PENINSULA,2.0TD,ACCIONA,2023-07-01,Poniente,0.218279,0.171470,0.148202,0.0,0.0,0.0
4,PENINSULA,2.0TD,ACCIONA,2023-07-01,Poniente+,0.218279,0.171470,0.148202,0.0,0.0,0.0


In [8]:
filas_mas_cercanas.dropna(axis=0,inplace=True)

In [9]:
index_power = pd.read_csv('../data/processed/indexed_price_power.csv')

In [10]:
# condiciones para filtrar (sería con el desplegable)
condiciones_sistema_tarifa = (index_power['SISTEMA'] == 'PENINSULA') & (index_power['TARIFA'] == '2.0TD') & (index_power['PRODUCTO'] == 'INDEXADO')

condiciones_cias = index_power['CIA'].isin(['ACCIONA', 'AEQ', 'CANDELA', 'FACTOR', 'IGNIS', 'MAX'])

index_power_filtrado = index_power[condiciones_sistema_tarifa & condiciones_cias] # aplicar el filtro

index_power_filtrado.head()

,SISTEMA,CIA,PRODUCTO,PRODUCTO_CIA,TARIFA,P1,P2,P3,P4,P5,P6
94,PENINSULA,ACCIONA,INDEXADO,CIERZO,2.0TD,0.088121,0.019570,0.0,0.0,0.0,0.0
98,PENINSULA,ACCIONA,INDEXADO,LEVANTE+,2.0TD,0.082641,0.014091,0.0,0.0,0.0,0.0
102,PENINSULA,ACCIONA,INDEXADO,LEVANTE,2.0TD,0.071682,0.003132,0.0,0.0,0.0,0.0
106,PENINSULA,ACCIONA,INDEXADO,PONIENTE+,2.0TD,0.077162,0.008611,0.0,0.0,0.0,0.0
110,PENINSULA,ACCIONA,INDEXADO,PONIENTE,2.0TD,0.071682,0.003132,0.0,0.0,0.0,0.0


In [11]:
filas_mas_cercanas['FEE'] = filas_mas_cercanas['FEE'].str.upper()

In [12]:
# funciones de cálculo importes
def calcular_energia(cons_mens_P1,cons_mens_P2,cons_mens_P3, precio_mens_P1,precio_mens_P2,precio_mens_P3,descuento):
    # sumatorio_cons_mens=cons_mens_P1+cons_mens_P2+cons_mens_P3
    precio_P1_descuento= precio_mens_P1 * (1-descuento) #€
    precio_P2_descuento= precio_mens_P2 * (1-descuento)
    precio_P3_descuento= precio_mens_P3 * (1-descuento)
    
    #añadir P4,P5,P6 como None/0/1

    total_pago_P1_energia= cons_mens_P1 * precio_P1_descuento #€
    total_pago_P2_energia= cons_mens_P2 * precio_P2_descuento
    total_pago_P3_energia= cons_mens_P3 * precio_P3_descuento

    sumatorio_total_pago_energia = total_pago_P1_energia + total_pago_P2_energia + total_pago_P3_energia
    return sumatorio_total_pago_energia

def calcular_potencia(potencia_contratada_P1,potencia_contratada_P2,dias,precio_potencia_dia_P1,precio_potencia_dia_P2,dto_p):
    total_pago_P1_potencia= dias * precio_potencia_dia_P1 * potencia_contratada_P1 * (1-dto_p)
    total_pago_P2_potencia= dias * precio_potencia_dia_P2 * potencia_contratada_P2 * (1-dto_p)
    sumatorio_total_pago_potencia = total_pago_P1_potencia + total_pago_P2_potencia
    return sumatorio_total_pago_potencia
    
def calcular_total_factura(sumatorio_total_pago_energia,sumatorio_total_pago_potencia,impuesto_electrico,otros,alquiler_equipo,IVA):
    bi_IVA= (sumatorio_total_pago_energia + sumatorio_total_pago_potencia
         +impuesto_electrico + otros + alquiler_equipo)
    importe_total_factura_mens= bi_IVA * (1+IVA)
    return importe_total_factura_mens


In [13]:
def encontrar_opcion_mas_barata(df_energia, df_potencia):
    opciones = []

    df_combinado = pd.merge(df_energia, df_potencia, on='CIA',suffixes=["_E","_P"])
    df_combinado.dropna(axis=0,inplace=True)

    cons_mens_P1, cons_mens_P2, cons_mens_P3 = 74, 83, 168 #CAMBIAR A INPUT DE LA FACTURA
    potencia_contratada_P1, potencia_contratada_P2 = 3.45, 3.45 #CAMBIAR A INPUT DE LA FACTURA
    precio_mens_P1, precio_mens_P2, precio_mens_P3 = 0.128515, 0.128515, 0.128515
    precio_potencia_dia_P1, precio_potencia_dia_P2 = 0.09968, 0.033816
    descuento = 0.02  # CAMBIAR A INPUT DE LA FACTURA
    dias = 33 # CAMBIAR A INPUT DE LA FACTURA
    dto_p = 0.0
    
    sumatorio_total_pago_energia = calcular_energia(cons_mens_P1, cons_mens_P2, cons_mens_P3, precio_mens_P1, precio_mens_P2, precio_mens_P3, descuento)
    sumatorio_total_pago_potencia = calcular_potencia(potencia_contratada_P1, potencia_contratada_P2, dias, precio_potencia_dia_P1, precio_potencia_dia_P2,dto_p)
    impuesto_electrico, otros, alquiler_equipo, IVA = 0.33, 1.27, 0.88,0.05  # CAMBIAR A INPUT DE LA FACTURA

    # print(sumatorio_total_pago_energia,sumatorio_total_pago_potencia)

    importe_total_factura_mens_actual = round(calcular_total_factura(sumatorio_total_pago_energia, sumatorio_total_pago_potencia, impuesto_electrico, otros,alquiler_equipo, IVA),2)





    for index,row in df_combinado.iterrows():
        cons_mens_P1, cons_mens_P2, cons_mens_P3 = 74, 83, 168 #CAMBIAR A INPUT DE LA FACTURA
        potencia_contratada_P1, potencia_contratada_P2 = 3.45, 3.45 #CAMBIAR A INPUT DE LA FACTURA
        precio_mens_P1, precio_mens_P2, precio_mens_P3 = row['P1_E'], row['P2_E'], row['P3_E']
        precio_potencia_dia_P1, precio_potencia_dia_P2 = row['P1_P'], row['P2_P']
        descuento = 0.02  # CAMBIAR A INPUT DE LA FACTURA
        dias = 33 # CAMBIAR A INPUT DE LA FACTURA
        dto_p = 0.0
        
        sumatorio_total_pago_energia = calcular_energia(cons_mens_P1, cons_mens_P2, cons_mens_P3, precio_mens_P1, precio_mens_P2, precio_mens_P3, descuento)
        sumatorio_total_pago_potencia = calcular_potencia(potencia_contratada_P1, potencia_contratada_P2, dias, precio_potencia_dia_P1, precio_potencia_dia_P2,dto_p)
        impuesto_electrico, otros, alquiler_equipo, IVA = 0.33, 1.27, 0.88,0.05  # CAMBIAR A INPUT DE LA FACTURA

        # print(sumatorio_total_pago_energia,sumatorio_total_pago_potencia)

        importe_total_factura_mens = round(calcular_total_factura(sumatorio_total_pago_energia, sumatorio_total_pago_potencia, impuesto_electrico, otros,alquiler_equipo, IVA),2)

        opciones.append({
            'CIA': row['CIA'],
            'FEE': row['FEE'],
            'PRODUCTO_CIA': row['PRODUCTO_CIA'],
            'CostoTotal': importe_total_factura_mens
        })

    # Opción más barata para cada compañía
    df_opciones = pd.DataFrame(opciones)
    idx_opcion_mas_barata = df_opciones['CostoTotal'].idxmin()
    
    opcion_barata=df_opciones.iloc[idx_opcion_mas_barata]

    opciones_mas_baratas = df_opciones.nsmallest(5, 'CostoTotal')
    ahorro_euros=round(importe_total_factura_mens_actual-opcion_barata['CostoTotal'],2)
    porcentaje_ahorro= round((ahorro_euros/importe_total_factura_mens_actual)*100,2)

    return ('Precio actual:',importe_total_factura_mens_actual,'Opción más barata:',opcion_barata,
            'Ahorro:',ahorro_euros, 'Porcentaje de ahorro:',f"{porcentaje_ahorro:.1f}%","Opciones más baratas:", opciones_mas_baratas)

In [14]:
encontrar_opcion_mas_barata(filas_mas_cercanas,index_power_filtrado)

('Precio actual:',
 61.54,
 'Opción más barata:',
 CIA                         FACTOR
 FEE                              4
 PRODUCTO_CIA    DOMESTICO_POOLDBOE
 CostoTotal                   62.43
 Name: 128, dtype: object,
 'Ahorro:',
 -0.89,
 'Porcentaje de ahorro:',
 '-1.4%',
 'Opciones más baratas:',
         CIA  FEE         PRODUCTO_CIA  CostoTotal
 128  FACTOR    4   DOMESTICO_POOLDBOE       62.43
 130  FACTOR    4  NEGOCIO_POOLNAHORRO       62.43
 135  FACTOR    6   DOMESTICO_POOLDBOE       63.09
 137  FACTOR    6  NEGOCIO_POOLNAHORRO       63.09
 142  FACTOR  7.5   DOMESTICO_POOLDBOE       63.60)

In [15]:


def encontrar_opcion_mas_barata_cia(df_energia, df_potencia):
    opciones = []

    df_combinado = pd.merge(df_energia, df_potencia, on='CIA', suffixes=["_E", "_P"])
    df_combinado.dropna(axis=0, inplace=True)

    cons_mens_P1, cons_mens_P2, cons_mens_P3 = 74, 83, 168  # CAMBIAR A INPUT DE LA FACTURA
    potencia_contratada_P1, potencia_contratada_P2 = 3.45, 3.45  # CAMBIAR A INPUT DE LA FACTURA
    precio_mens_P1, precio_mens_P2, precio_mens_P3 = 0.128515, 0.128515, 0.128515
    precio_potencia_dia_P1, precio_potencia_dia_P2 = 0.09968, 0.033816
    descuento = 0.02  # CAMBIAR A INPUT DE LA FACTURA
    dias = 33  # CAMBIAR A INPUT DE LA FACTURA
    dto_p = 0.0

    sumatorio_total_pago_energia = calcular_energia(cons_mens_P1, cons_mens_P2, cons_mens_P3, precio_mens_P1,
                                                    precio_mens_P2, precio_mens_P3, descuento)
    sumatorio_total_pago_potencia = calcular_potencia(potencia_contratada_P1, potencia_contratada_P2, dias,
                                                      precio_potencia_dia_P1, precio_potencia_dia_P2,dto_p)
    impuesto_electrico, otros, alquiler_equipo, IVA = 0.33, 1.27, 0.88, 0.05  # CAMBIAR A INPUT DE LA FACTURA

    importe_total_factura_mens_actual = round(calcular_total_factura(sumatorio_total_pago_energia, sumatorio_total_pago_potencia, impuesto_electrico,
                               otros, alquiler_equipo, IVA), 2)

    # Create a dictionary to store the minimum cost for each 'CIA'
    min_cost_dict = {}

    for index, row in df_combinado.iterrows():
        cons_mens_P1, cons_mens_P2, cons_mens_P3 = 74, 83, 168  # CAMBIAR A INPUT DE LA FACTURA
        potencia_contratada_P1, potencia_contratada_P2 = 3.45, 3.45  # CAMBIAR A INPUT DE LA FACTURA
        precio_mens_P1, precio_mens_P2, precio_mens_P3 = row['P1_E'], row['P2_E'], row['P3_E']
        precio_potencia_dia_P1, precio_potencia_dia_P2 = row['P1_P'], row['P2_P']
        descuento = 0.02  # CAMBIAR A INPUT DE LA FACTURA
        dias = 33  # CAMBIAR A INPUT DE LA FACTURA
        dto_p = 0.0

        sumatorio_total_pago_energia = calcular_energia(cons_mens_P1, cons_mens_P2, cons_mens_P3, precio_mens_P1,
                                                        precio_mens_P2, precio_mens_P3, descuento)
        sumatorio_total_pago_potencia = calcular_potencia(potencia_contratada_P1, potencia_contratada_P2, dias,
                                                          precio_potencia_dia_P1, precio_potencia_dia_P2,dto_p)
        impuesto_electrico, otros, alquiler_equipo, IVA = 0.33, 1.27, 0.88, 0.05  # CAMBIAR A INPUT DE LA FACTURA

        importe_total_factura_mens = round(calcular_total_factura(sumatorio_total_pago_energia, sumatorio_total_pago_potencia, impuesto_electrico,
                                   otros, alquiler_equipo, IVA), 2)

        # Update the minimum cost for each 'CIA'
        if row['CIA'] not in min_cost_dict or importe_total_factura_mens < min_cost_dict[row['CIA']]:
            min_cost_dict[row['CIA']] = importe_total_factura_mens

   
    # Create a list with the minimum cost entries for each 'CIA' and calculate savings for each option
    opciones = [{
        'CIA': cia,
        'FEE': df_combinado.loc[df_combinado['CIA'] == cia, 'FEE'].values[0],
        'PRODUCTO_CIA': df_combinado.loc[df_combinado['CIA'] == cia, 'PRODUCTO_CIA'].values[0],
        'CostoTotal': min_cost,
        'Ahorro': round(importe_total_factura_mens_actual - min_cost, 2),
        'PorcentajeAhorro': round(((importe_total_factura_mens_actual - min_cost) / importe_total_factura_mens_actual) * 100, 2)
    } for cia, min_cost in min_cost_dict.items()]

    # Convert the list to a DataFrame
    df_opciones = pd.DataFrame(opciones)

    # Opción más barata para cada compañía
    opciones_mas_baratas = df_opciones.nsmallest(5, 'CostoTotal')

    return ('Precio actual:', importe_total_factura_mens_actual, 'Opciones más baratas:', opciones_mas_baratas)





In [16]:
encontrar_opcion_mas_barata_cia(filas_mas_cercanas,index_power_filtrado)

('Precio actual:',
 61.54,
 'Opciones más baratas:',
        CIA               FEE        PRODUCTO_CIA  CostoTotal  Ahorro  \
 3   FACTOR                 4  DOMESTICO_POOLDBOE       62.43   -0.89   
 4    IGNIS   30 MARE PLUS 1           MARE KIT 1       64.34   -2.80   
 0  ACCIONA            CIERZO              CIERZO       65.38   -3.84   
 2  CANDELA      01 / BI0,035              AMPERE       69.23   -7.69   
 1      AEQ                 3             ARMONIA       69.68   -8.14   
 
    PorcentajeAhorro  
 3             -1.45  
 4             -4.55  
 0             -6.24  
 2            -12.50  
 1            -13.23  )